In [17]:
import pandas as pd

import mysql.connector

from sqlalchemy import create_engine

from skimpy import skim

# 0- importation des données en df

In [2]:
username = 'postgres'
password = 'admin'
host = '127.0.0.1'  # ou l'adresse IP de votre serveur PostgreSQL
port = '5432'  # le port par défaut pour PostgreSQL est 5432
database = 'preventive_maintenance'

# Créer l'URL de connexion
db_url = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'

# Créer le moteur SQLAlchemy
engine = create_engine(db_url)

# Se connecter à la base de données
connexion = engine.connect()


In [3]:
df_aero = pd.read_sql_query('''
                  SELECT * 
                  FROM aeronefs
                  ''', connexion)


In [4]:
df_composants = pd.read_sql_query('''
                  SELECT * 
                  FROM composants
                  ''', connexion)

In [5]:
df_degradations = pd.read_sql_query('''
                  SELECT * 
                  FROM degradations
                  ''', connexion)

In [6]:
df_logs_vols = pd.read_sql_query('''
                  SELECT * 
                  FROM logs_vols
                  ''', connexion)

In [7]:
connexion.close()

# 1- merge des dataframes

In [8]:
# merge du aeronefs et compoosants en left join avec aeronefs en left

df_merge_aero_log = pd.merge(df_aero, df_logs_vols, how='inner', left_on='ref_aero', right_on='aero_linked')

In [9]:
# merge du df_merge_aero_compo et degradations en left join avec df_merge_aero_compo en left

df_merge_aero_log_degrad = pd.merge(df_merge_aero_log, df_degradations, how='inner', left_on='ref_aero', right_on='linked_aero')

In [10]:
# merge du df_merge_aero_compo et degradations en left join avec df_merge_aero_compo en left

df_merge_total = pd.merge(df_merge_aero_log_degrad, df_composants, how='left', left_on='compo_concerned', right_on='ref_compo')

In [11]:
df_merge_total.shape

(5140440, 32)

# 2- Nettoayage du df

In [12]:
df_merge_total.columns

Index(['ref_aero', 'type_model', 'debut_service', 'last_maint',
       'en_maintenance', 'end_maint', 'ref_vol', 'aero_linked', 'jour_vol',
       'time_en_air', 'sensor_data', 'etat_voyant', 'temp', 'temp_unit',
       'pressure', 'pressure_unit', 'vibrations', 'vibrations_unit', 'clef',
       'ref_deg', 'linked_aero', 'compo_concerned', 'usure_nouvelle',
       'measure_day', 'need_replacement', 'ref_compo', 'categorie', 'aero',
       'desc_compo', 'lifespan', 'taux_usure_actuel', 'cout'],
      dtype='object')

In [13]:
df_merge_total = df_merge_total.drop(columns=[
                            'aero_linked', 
                            'sensor_data',
                            'temp_unit',
                            'pressure_unit', 
                            'vibrations_unit',
                            'linked_aero', 
                            'compo_concerned',
                            'aero'])

In [15]:
pd.set_option("display.max_columns", None)

In [18]:
df_merge_total.head(3)

,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint,ref_vol,jour_vol,time_en_air,etat_voyant,temp,pressure,vibrations,clef,ref_deg,usure_nouvelle,measure_day,need_replacement,ref_compo,categorie,desc_compo,lifespan,taux_usure_actuel,cout
0,B767_2803,B767,2020-12-03,2024-06-01,False,NaT,V04670188,2024-06-04,0.6,0,4.6,896.2,1.72443,D003716B767_2803REAB767-B767_2803-02024-06-04,D003716,5.241075,2024-06-04,False,REAB767-B767_2803-0,Composants Critiques,Réacteur gauche,13045,4.184430,15552
1,B767_2803,B767,2020-12-03,2024-06-01,False,NaT,V04670188,2024-06-04,0.6,0,4.6,896.2,1.72443,D004900B767_2803REAB767-B767_2803-12024-06-04,D004900,20.329510,2024-06-04,False,REAB767-B767_2803-1,Composants Critiques,Réacteur droit,13626,19.225804,17559
2,B767_2803,B767,2020-12-03,2024-06-01,False,NaT,V04670188,2024-06-04,0.6,0,4.6,896.2,1.72443,D009419B767_2803SYSB767-B767_2803-22024-06-04,D009419,47.413035,2024-06-04,False,SYSB767-B767_2803-2,Composants Critiques,Système de navigation,10541,46.559214,16514


In [19]:
skim(df_merge_total)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                 Data Types                                                                │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                         │
│ ┃ dataframe         ┃ Values  ┃ ┃ Column Type ┃ Count ┃                                                         │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                         │
│ │ Number of rows    │ 5140440 │ │ string      │ 8     │                                                         │
│ │ Number of columns │ 24      │ │ float64     │ 6     │                                                         │
│ └───────────────────┴─────────┘ │ object      │ 4     │                                                         │
│                                 │ int32       │ 3     │                                                         │
│                                 │ bool        │ 2     │                                                         │
│                                 │ datetime64  │ 1     │                                                         │
│                                 └─────────────┴───────┘                                                         │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name         ┃ NA  ┃ NA %  ┃ mean    ┃ sd      ┃ p0        ┃ p25   ┃ p50   ┃ p75   ┃ p100  ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━┩  │
│ │ time_en_air         │   0 │     0 │   5.059 │   2.444 │       0.6 │   3.3 │     5 │   6.7 │    15 │  ▅▇▇▃  │  │
│ │ etat_voyant         │   0 │     0 │  0.1796 │  0.4412 │         0 │     0 │     0 │     0 │     3 │  ▇ ▁   │  │
│ │ temp                │   0 │     0 │   0.114 │   9.867 │     -36.5 │  -6.5 │   0.2 │   6.7 │    35 │  ▂▇▇▂  │  │
│ │ pressure            │   0 │     0 │   950.8 │   49.23 │     757.3 │ 917.2 │ 951.8 │   985 │  1110 │  ▁▅▇▃  │  │
│ │ vibrations          │   0 │     0 │   2.518 │   1.451 │  0.001847 │ 1.271 │ 2.525 │ 3.771 │ 4.999 │ ▇▇▇▇▇▇ │  │
│ │ usure_nouvelle      │   0 │     0 │   38.82 │   18.16 │      0.11 │ 23.77 │ 38.95 │ 53.71 │   100 │ ▃▇▇▇▁  │  │
│ │ lifespan            │   0 │     0 │   13570 │    8481 │     10020 │ 11020 │ 12640 │ 13630 │ 68980 │   ▇    │  │
│ │ taux_usure_actuel   │   0 │     0 │   31.16 │   17.26 │      0.11 │  16.4 │ 31.23 │ 45.88 │ 68.91 │ ▇▇▇▇▇▂ │  │
│ │ cout                │   0 │     0 │   11420 │    5763 │      2021 │  4801 │ 12660 │ 16270 │ 19490 │ ▇▁▂▇▅▇ │  │
│ └─────────────────────┴─────┴───────┴─────────┴─────────┴───────────┴───────┴───────┴───────┴───────┴────────┘  │
│                                                      bool                                                       │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                              ┃ true              ┃ true rate                ┃ hist             ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩  │
│ │ en_maintenance                           │            277065 │                    0.054 │        ▇         │  │
│ │ need_replacement                         │                 0 │                        0 │          ▇       │  │
│ └──────────────────────────────────────────┴───────────────────┴──────────────────────────┴──────────────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━

In [23]:
# suppression des colonne n'apportant pas d'informations
df_merge_total = df_merge_total.drop(columns=['need_replacement', 'end_maint'])

In [22]:
# modification du type pour date 
df_merge_total['debut_service'] = pd.to_datetime(df_merge_total['debut_service'], format='%Y-%m-%d')
df_merge_total['last_maint'] = pd.to_datetime(df_merge_total['last_maint'], format='%Y-%m-%d')
df_merge_total['jour_vol'] = pd.to_datetime(df_merge_total['jour_vol'], format='%Y-%m-%d')
df_merge_total['measure_day'] = pd.to_datetime(df_merge_total['measure_day'], format='%Y-%m-%d')

In [25]:
# modif type category
df_merge_total['etat_voyant'] = df_merge_total['etat_voyant'].astype('category')         
df_merge_total['categorie'] = df_merge_total['categorie'].astype('category') 

In [ ]:
# Ajout d'une colonne temps de vol cumulé depuis la derniere maintenance


In [26]:
skim(df_merge_total)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                 Data Types               Categories                                       │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓ ┏━━━━━━━━━━━━━━━━━━━━━━━┓                               │
│ ┃ dataframe         ┃ Values  ┃ ┃ Column Type ┃ Count ┃ ┃ Categorical Variables ┃                               │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩ ┡━━━━━━━━━━━━━━━━━━━━━━━┩                               │
│ │ Number of rows    │ 5140440 │ │ string      │ 7     │ │ etat_voyant           │                               │
│ │ Number of columns │ 22      │ │ float64     │ 6     │ │ categorie             │                               │
│ └───────────────────┴─────────┘ │ datetime64  │ 4     │ └───────────────────────┘                               │
│                                 │ category    │ 2     │                                                         │
│                                 │ int32       │ 2     │                                                         │
│                                 │ bool        │ 1     │                                                         │
│                                 └─────────────┴───────┘                                                         │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name         ┃ NA  ┃ NA %  ┃ mean   ┃ sd     ┃ p0        ┃ p25    ┃ p50    ┃ p75   ┃ p100  ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━┩  │
│ │ time_en_air         │   0 │     0 │  5.059 │  2.444 │       0.6 │    3.3 │      5 │   6.7 │    15 │  ▅▇▇▃  │  │
│ │ temp                │   0 │     0 │  0.114 │  9.867 │     -36.5 │   -6.5 │    0.2 │   6.7 │    35 │  ▂▇▇▂  │  │
│ │ pressure            │   0 │     0 │  950.8 │  49.23 │     757.3 │  917.2 │  951.8 │   985 │  1110 │  ▁▅▇▃  │  │
│ │ vibrations          │   0 │     0 │  2.518 │  1.451 │  0.001847 │  1.271 │  2.525 │ 3.771 │ 4.999 │ ▇▇▇▇▇▇ │  │
│ │ usure_nouvelle      │   0 │     0 │  38.82 │  18.16 │      0.11 │  23.77 │  38.95 │ 53.71 │   100 │ ▃▇▇▇▁  │  │
│ │ lifespan            │   0 │     0 │  13570 │   8481 │     10020 │  11020 │  12640 │ 13630 │ 68980 │   ▇    │  │
│ │ taux_usure_actuel   │   0 │     0 │  31.16 │  17.26 │      0.11 │   16.4 │  31.23 │ 45.88 │ 68.91 │ ▇▇▇▇▇▂ │  │
│ │ cout                │   0 │     0 │  11420 │   5763 │      2021 │   4801 │  12660 │ 16270 │ 19490 │ ▇▁▂▇▅▇ │  │
│ └─────────────────────┴─────┴───────┴────────┴────────┴───────────┴────────┴────────┴───────┴───────┴────────┘  │
│                                                    category                                                     │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                      ┃ NA        ┃ NA %           ┃ ordered               ┃ unique             ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩  │
│ │ etat_voyant                      │         0 │              0 │ False                 │                  4 │  │
│ │ categorie                        │         0 │              0 │ False                 │                  3 │  │
│ └──────────────────────────────────┴───────────┴────────────────┴───────────────────────┴────────────────────┘  │
│                                                      bool                                                       │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                            ┃ true       